In [5]:
import asf_search as asf
import geopandas as gpd
import xarray as xr
import pandas as pd
import rasterio as rio
import rioxarray

In [6]:
# open processing decisions csv
md_df = pd.read_csv('md_processing_decisions.csv').iloc[:, 0:9]
md_df['survey_name'] = md_df['survey_name'].str.strip()

In [7]:
# download OPERA layover and shadow mask
def download_mask(burst_name, rm_downloads=True):
    # query asf for burst
    results = asf.search(dataset='OPERA-S1',
                         processingLevel='RTC',
                         operaBurstID=[f'T{burst_name}'])
    gf = gpd.GeoDataFrame.from_features(results.geojson(), crs=4326)
    # find mask url
    mask_url = [x for x in gf.additionalUrls[0] if x.endswith('mask.tif')][0]
    # grab mask fn
    mask_fn = mask_url.split('/')[-1]
    # download file to local tmp dir
    asf.download_url(mask_url, './tmp')
    # open in xarray
    mask_ds = xr.open_dataarray(f"./tmp/{mask_fn}")
    # separate based on values
    ds = mask_ds.expand_dims(dict(mask=['valid','shadow','layover','layover_and_shadow','exterior'])).to_dataset('mask')

    ds['valid'] = mask_ds==0
    ds['shadow'] = mask_ds==1
    ds['layover'] = mask_ds==2
    ds['layover_and_shadow'] = mask_ds==3
    ds['exterior'] = mask_ds==255

    # remove tmp download
    if rm_downloads == True:
        !rm tmp/$mask_fn

    return ds.squeeze()

In [8]:
for i, row in md_df.iterrows():
    print(row.survey_name)
    asc_mask = download_mask(row.asc_burst)
    asc_ds = xr.open_dataset(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name}_asc_median_velocity.tif').squeeze()
    asc_mask = asc_mask.astype(int).rio.reproject_match(asc_ds)
    asc_mask.valid.astype('float32').rio.to_raster(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name}_asc_valid.tif')
    
    des_mask = download_mask(row.des_burst)
    des_ds = xr.open_dataset(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name.strip()}_des_median_velocity.tif').squeeze()
    des_mask = des_mask.astype(int).rio.reproject_match(des_ds)
    des_mask.valid.astype('float32').rio.to_raster(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name}_des_valid.tif')
    # combine for a single valid mask for ud/ew
    ((asc_mask.valid + des_mask.valid) == 2).astype('int').astype('float32').rio.to_raster(f'/home/jovyan/repos/incubator_moraine_dam_analysis/dams/{row.survey_name}/{row.survey_name}_combined_valid.tif')

    

rolpa
barun
imja
lumding
chamlang_south
thugali
nagma
chamlang_north
muli_tal
barun_pokhari
east_hongu_1
birendra
mera
upper_barun
gakal_gl_0008
kotam_gl_0111
east_hongu_2
tallo_kekyap
kekyap
dig
sabai
kodud_gl_0205
hongu_1
